# Analyse des Avis et Alertes ANSSI avec Enrichissement des CVE

## Étape 1 : Extraction des Flux RSS

In [ ]:
# faire "pip install feedparser" si besoin dans le terminal 
import feedparser
url = "https://www.cert.ssi.gouv.fr/avis/feed"
rss_feed = feedparser.parse(url)
for entry in rss_feed.entries:
    print("Titre :", entry.title)
    print("Description:", entry.description)
    print("Lien :", entry.link)
    print("Date :", entry.published)

Titre : Multiples vulnérabilités dans Mattermost Server (13 mai 2025)
Description: De multiples vulnérabilités ont été découvertes dans Mattermost Server. Elles permettent à un attaquant de provoquer une atteinte à la confidentialité des données et un contournement de la politique de sécurité.
Lien : https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0392/
Date : Tue, 13 May 2025 00:00:00 +0000
Titre : Multiples vulnérabilités dans les produits SAP (13 mai 2025)
Description: De multiples vulnérabilités ont été découvertes dans les produits SAP. Certaines d'entre elles permettent à un attaquant de provoquer une exécution de code arbitraire à distance, une atteinte à la confidentialité des données et une injection de code indirecte à distance (XSS).
Lien : https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0396/
Date : Tue, 13 May 2025 00:00:00 +0000
Titre : Vulnérabilité dans Roundcube (02 juin 2025)
Description: Une vulnérabilité a été découverte dans Roundcube Roundcube Webmail. Elle pe

## Étape 2 : Extraction des CVE

In [4]:
import requests
import re
url = "https://www.cert.ssi.gouv.fr/alerte/CERTFR-2024-ALE-001/json/"
response = requests.get(url)
data = response.json()
#Extraction des CVE reference dans la clé cves du dict data
ref_cves=list(data["cves"])
#attention il s’agit d’une liste des dictionnaires avec name et url comme clés
print( "CVE référencés ", ref_cves)
# Extraction des CVE avec une regex
cve_pattern = r"CVE-\d{4}-\d{4,7}"
cve_list = list(set(re.findall(cve_pattern, str(data))))
print("CVE trouvés :", cve_list)

CVE référencés  [{'name': 'CVE-2023-46805', 'url': 'https://www.cve.org/CVERecord?id=CVE-2023-46805'}, {'name': 'CVE-2024-21887', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-21887'}, {'name': 'CVE-2024-21893', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-21893'}, {'name': 'CVE-2024-21888', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-21888'}, {'name': 'CVE-2024-22024', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-22024'}]
CVE trouvés : ['CVE-2024-21893', 'CVE-2024-21888', 'CVE-2024-22024', 'CVE-2024-21887', 'CVE-2023-46805']


## Étape 3 : Enrichissement des CVE

### Exemple de connexion à l'API CVE :

In [5]:
# import requests
cve_id = "CVE-2023-24488"
url = f"https://cveawg.mitre.org/api/cve/{cve_id}"
response = requests.get(url)
data = response.json()

# Extraire la description
description = data["containers"]["cna"]["descriptions"][0]["value"]
# Extraire le score CVSS
#ATTENTION tous les CVE ne contiennent pas nécessairement ce champ, gérez l’exception,
#ou peut etre au lieu de cvssV3_0 c’est cvssV3_1 ou autre clé
cvss_score =data["containers"]["cna"]["metrics"][0]["cvssV3_1"]["baseScore"]

cwe = "Non disponible"
cwe_desc="Non disponible"
problemtype = data["containers"]["cna"].get("problemTypes", {})
if problemtype and "descriptions" in problemtype[0]:
    cwe = problemtype[0]["descriptions"][0].get("cweId","Non disponible")
    cwe_desc=problemtype[0]["descriptions"][0].get("description","Non disponible")

# Extraire les produits affectés
affected = data["containers"]["cna"]["affected"]
for product in affected:
    vendor = product["vendor"]
    product_name = product["product"]
    versions = [v["version"] for v in product["versions"] if v["status"] == "affected"]
    print(f"Éditeur : {vendor}, Produit : {product_name}, Versions : {','.join(versions)}")

# Afficher les résultats
print(f"CVE : {cve_id}")
print(f"Description : {description}")
print(f"Score CVSS : {cvss_score}")
print(f"Type CWE : {cwe}")
print(f"CWE Description : {cwe_desc}")

Éditeur : Citrix, Produit : Citrix ADC and Citrix Gateway , Versions : 13.1,13.0,12.1,12.1-FIPS ,13.1-FIPS ,12.1-NDcPP
CVE : CVE-2023-24488
Description : Cross site scripting vulnerability in Citrix ADC and Citrix Gateway  in allows and attacker to perform cross site scripting
Score CVSS : 6.1
Type CWE : CWE-79
CWE Description : CWE-79 Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting')


### Exemple de connexion à l’API EPSS:

In [6]:
import requests
# URL de l'API EPSS pour récupérer la probabilité d'exploitation
cve_id = "CVE-2023-46805"
url = f"https://api.first.org/data/v1/epss?cve={cve_id}"
# Requête GET pour récupérer les données JSON
response = requests.get(url)
data = response.json()
# Extraire le score EPSS
epss_data = data.get("data", [])
if epss_data:
    epss_score = epss_data[0]["epss"]
    print(f"CVE : {cve_id}")
    print(f"Score EPSS : {epss_score}")
else:
    print(f"Aucun score EPSS trouvé pour {cve_id}")

CVE : CVE-2023-46805
Score EPSS : 0.943980000
